In [ ]:
######################################################################
# Modelling Sleep Duration Using Gaussian Processes
# LE49: MiniProject
# Jan Ondras (jo356), Trinity College
# 2017/2018
######################################################################
####################################################################################
# For each user fit GP to sleepduration data and save extracted kernel parameters to: 'Params/' + t
####################################################################################


import numpy as np
import os
import time
import glob
from datetime import datetime, timedelta
import matplotlib.pyplot as plt         # MATLAB-style plotting
import sklearn.gaussian_process as gp   # Gaussian process modeling
from sklearn.gaussian_process.kernels import ExpSineSquared, WhiteKernel, ConstantKernel, RBF, Matern

# log_transform_y = False # as investigated, this is not the case
y_norm = False
GLOBAL_MEAN_SD = 7.26 # Global mean sleep duration (prior), substracted from all timeseries

t = 'sleepduration' # type of data to extract
min_num_measurements = 300 #358 # 300 for 1266, 340 for 536, 350 for 305
yeardays = np.arange(366)

kernel_types = ['1_x1264', '3_x1264']  #1.) PERIODIC KERNEL, # 3.) MATERN * PERIODIC KERNEL, 1264 users
# if log_transform_y:
#     kernel_types = [x+'_log' for x in kernel_types]

nu = 2.5

seed=37
NR = 96 # number of restarts of initialisation

types = {
    'sleepduration': '10',
    'bedin': '11',
    'bedout': '12',
    'steps': '1',
    'weight': '2',
    'bloodpressure': '4',
    'heartrate': '7'
}
for kernel_type in kernel_types:
    os.mkdir('./../Dataset/Params/clean_' + t + '_' + kernel_type)

def fitGPsave(X, y, kernel, kernel_type):
    ##################################################################################
    # Fit GP
    model = gp.GaussianProcessRegressor(kernel=kernel, random_state=seed, 
                                        n_restarts_optimizer=NR, normalize_y=y_norm).fit(X, y)            
    #print "\tBest params: ", np.exp(model.kernel_.theta).tolist()
    #print "\tLog marginal likelihood: ", model.log_marginal_likelihood_value_
    
    ##################################################################################
    # Predict
    mean, std = model.predict(yeardays[..., np.newaxis], return_std=True)
       
    #################################################################################
    # Save actual values and log-transformed parameters
    # and all predictions with stds
    np.savez('./../Dataset/Params/clean_' + t + '_' + kernel_type + '/' + UID + '.npz', 
             params=np.exp(model.kernel_.theta), params_log=model.kernel_.theta, 
            pred_mean=mean.squeeze(), pred_std=std)

st = time.time()
# For each user
for file_name in glob.glob('./../Dataset/clean_' + t + '/*.npz'):
    UID = file_name.split('/')[-1][:-4]
    data = np.load(file_name)['xy']
    
    if len(data) < min_num_measurements:
        continue

    X = np.reshape(data[:,0], (-1,1))
    y = np.reshape(data[:,1], (-1,1))

    y = y - GLOBAL_MEAN_SD

#     if log_transform_y:
#         y = np.log(y)

    #1.) PERIODIC KERNEL
    kernel = ( ConstantKernel(constant_value=1.0, constant_value_bounds=(0.001, 100.)) 
            * ExpSineSquared(length_scale=1.0, periodicity=7.0, 
                             length_scale_bounds=(0.5, 365.0), periodicity_bounds=(1., 365.)) 
              + WhiteKernel(noise_level=1.0, noise_level_bounds=(0.01, 10.0)) )
    fitGPsave(X, y, kernel, kernel_types[0])
    
    # 3.) MATERN * PERIODIC KERNEL
    kernel = ( ConstantKernel(constant_value=1.0, constant_value_bounds=(0.001, 100.)) 
            * Matern(length_scale=1.0, length_scale_bounds=(0.5, 1000.0), nu=nu) 
            * ExpSineSquared(length_scale=1.0, periodicity=7.0, 
                             length_scale_bounds=(0.5, 365.0), periodicity_bounds=(1., 365.)) 
              + WhiteKernel(noise_level=1.0, noise_level_bounds=(0.01, 10.0)) )
    fitGPsave(X, y, kernel, kernel_types[1])

################################## NOT USED: OTHER KERNEL VARIATIONS
    # 4.)
#     kernel = ( ConstantKernel(constant_value=64.0, constant_value_bounds=(4, 144.)) 
#             * ExpSineSquared(length_scale=1.0, periodicity=7.0, 
#                             length_scale_bounds=(1., 7.0), periodicity_bounds=(7., 7.)) 
#              + WhiteKernel(noise_level=1.0, noise_level_bounds=(0.01, 10.0)) )
#     fitGPsave(X, y, kernel, kernel_types[0])
    
    # 5.)
#     kernel = ( ConstantKernel(constant_value=64.0, constant_value_bounds=(4, 144.)) 
#             * Matern(length_scale=7.0, length_scale_bounds=(7., 365.0), nu=2.5) 
#             * ExpSineSquared(length_scale=1.0, periodicity=7.0, 
#                              length_scale_bounds=(1., 7.0), periodicity_bounds=(7., 7.)) 
#               + WhiteKernel(noise_level=1.0, noise_level_bounds=(0.01, 10.0)) )
#     fitGPsave(X, y, kernel, kernel_types[1])
    
    # 6.)
#     kernel = ( ConstantKernel(constant_value=64.0, constant_value_bounds=(4, 144.)) 
#             * ExpSineSquared(length_scale=1.0, periodicity=7.0, 
#                             length_scale_bounds=(1., 365.0), periodicity_bounds=(1., 365.))
#              + WhiteKernel(noise_level=1.0, noise_level_bounds=(0.01, 10.0)) )
#     fitGPsave(X, y, kernel, kernel_types[0])
    
    # 7.)
#     kernel = ( ConstantKernel(constant_value=64.0, constant_value_bounds=(4, 144.)) 
#             * Matern(length_scale=7.0, length_scale_bounds=(7., 365.0), nu=2.5) 
#             * ExpSineSquared(length_scale=1.0, periodicity=7.0, 
#                              length_scale_bounds=(1., 365.0), periodicity_bounds=(1., 365.))
#               + WhiteKernel(noise_level=1.0, noise_level_bounds=(0.01, 10.0)) )
#     fitGPsave(X, y, kernel, kernel_types[3])


    print "Time taken: ", time.time()-st, (time.time()-st)/60. 

Time taken:  291.515192032 4.85858701468
Time taken:  598.473651886 9.97456163168
Time taken:  893.937535048 14.8989598672
Time taken:  1181.6498239 19.694164598
Time taken:  1437.25747204 23.9542918324
Time taken:  1746.01900697 29.100317351
Time taken:  2045.44376993 34.0907307982
Time taken:  2289.99809194 38.1666355491
Time taken:  2562.0967319 42.7016128977
Time taken:  2869.75618601 47.8292702834
Time taken:  3207.89440799 53.4649073998
Time taken:  3451.68126297 57.5280216177
Time taken:  3707.23375297 61.7872298161
Time taken:  3954.51286507 65.9085492174
Time taken:  4206.43247986 70.1072090983
Time taken:  4521.06866908 75.3511450847
Time taken:  4810.94033694 80.1823395014
Time taken:  5115.53708005 85.258952717
Time taken:  5400.77569699 90.012929066
Time taken:  5645.28690004 94.0881156842
Time taken:  5897.31463504 98.28857795
Time taken:  6218.27632308 103.637939668
Time taken:  6485.7584939 108.0959758
Time taken:  6740.020293 112.333672416
Time taken:  7069.72453904 11